In [3]:
import inro.modeller as m
import os
import glob
mm = m.Modeller()
eb = mm.emmebank
import inro.emme.matrix as _matrix
import pandas as pd
import numpy as np
from copy import deepcopy

matrix_calculator = m.Modeller().tool("inro.emme.matrix_calculation.matrix_calculator")
create_matrix =  m.Modeller().tool("inro.emme.data.matrix.create_matrix")
change_matrix = m.Modeller().tool("inro.emme.data.matrix.change_matrix_properties") 
import_matrices = mm.tool("tmg.input_output.import_binary_matrix")  
matrix_transaction = mm.tool('inro.emme.data.matrix.matrix_transaction')
create_extra = m.Modeller().tool("inro.emme.data.extra_attribute.create_extra_attribute")
network_calc= m.Modeller().tool("inro.emme.network_calculation.network_calculator")

def clear_matrices(emmebank, exception_array = [""], type = "FULL"):
    for matrix in emmebank.matrices():
        if matrix.type == type:
            if matrix.id not in exception_array:
                emmebank.delete_matrix(matrix.id)

def import_matrices_from_directory(matrix_folder, matrix_list, extension, scenario):
    #imports all matrices in directory and subfolders of the specified extension type
    #matrix_list should be in the format:
    #   {file_name1: matrix_num1, file_name2: matrix_num2, ...}
    import os

    for root, dirs, files in os.walk(matrix_folder):
        for matrix_file in files:
            if matrix_file.endswith(extension):
                name = os.path.splitext(matrix_file)[0]
                if name in matrix_list:
                    matrix_number = int(matrix_list[name][2:])
                    import_matrices(4, matrix_number ,os.path.join(root, matrix_file),scenario, name.replace("skim.","").replace("transit.","").replace("peak","pk").replace(" ","_") )
                    print "Imported matrix " + matrix_list[name]
                    change_matrix(matrix = matrix_list[name],
                        matrix_name = name.replace("skim.","").replace("transit.","").replace("peak","pk").replace(" ","_")[:40],
                        matrix_description = name)
    
    #check
    for name in matrix_list:
        if eb.matrix(matrix_list[name]): 
            n_matrix = eb.matrix(matrix_list[name]).get_numpy_data(scenario)
            if n_matrix.sum() == 0 :
                print "Matrix %s was not imported, or contains no values!" % name
        else:
            print "Matrix %s was not imported!" % name
    print "Finished Importing Matrices"
    
#obtain the minimum transit travel time between all modes.
#uses outputs from HDR transit time toolbox
mcSpec = {
    "expression": None,
    "result": "",
    "constraint": {
        "by_value": None,
        "by_zone": None
    },
    "aggregation": {
        "origins": None,
        "destinations": None
    },
    "type": "MATRIX_CALCULATION"
}
def minimum_transit_time(tt_dir, min_transit_time,scenario,go_sub_mtx_nums):
    
    #import the .311 files containing GO Train and subway OD travel times
    for root, dirs, files in os.walk(tt_dir):
        for matrix_file in files:
            if matrix_file.endswith(".311"):
                matrix_transaction(os.path.join(root, matrix_file),scenario)
                
    create_matrix(matrix_id = min_transit_time,
        matrix_name= "min.transit.time",
        matrix_description = "minimum.transit.time")

    matrix_list ={ "express.bus.actual": "",
                   "rapid.bus.actual":"",
                   "local.bus.actual":""
                   }

    #Import matrices
    for matrix_name in matrix_list.keys():
        matrix_list[matrix_name] = eb.available_matrix_identifier('FULL')
        create_matrix(matrix_id = matrix_list[matrix_name],
                      matrix_name= matrix_name,
                      matrix_description = matrix_name)

    import_matrices_from_directory(tt_dir, matrix_list, ".mdf", scenario)
    
    spec = {
           "type": "MATRIX_CALCULATION",
           "result": min_transit_time,
           "expression":  "(" + matrix_list["express.bus.actual"] + ".min."\
            + matrix_list["rapid.bus.actual"] + ".min."\
            + matrix_list["local.bus.actual"] + ")"
    }
    matrix_calculator(spec)

    for mtx_num in go_sub_mtx_nums:
        expression = min_transit_time + ".min.mf" + mtx_num + "*(mf" + mtx_num + "!=0)+" + min_transit_time + "*(mf" + mtx_num + "==0)"
        spec = {
           "type": "MATRIX_CALCULATION",
           "result": min_transit_time,
           "expression":  expression
        }
        matrix_calculator(spec)

    print "Finished Calculating OD Transit Travel Times"
    
def minimum_transit_time_mto(tt_dir,mto_skims, min_transit_time, scenario):            
    create_matrix(matrix_id = min_transit_time,
        matrix_name= "min.transit.time",
        matrix_description = "minimum.transit.time")

    matrix_list ={"express.bus.actual": "",
                   "rapid.bus.actual":"",
                   "local.bus.actual":""
                   }
    
    mat_list_mto = {"Express Bus": "",
                   "Local Bus": "",
                   "Rapid Bus": "",
                   "Subway": "",
                   "Train": ""}

    matching = {"Express Bus": "express.bus.actual",
               "Local Bus":"local.bus.actual",
               "Rapid Bus":"rapid.bus.actual"}
    #Import matrices
    for matrix_name in matrix_list.keys():
        matrix_list[matrix_name] = eb.available_matrix_identifier('FULL')
        create_matrix(matrix_id = matrix_list[matrix_name],
                      matrix_name= matrix_name,
                      matrix_description = matrix_name)

    import_matrices_from_directory(tt_dir, matrix_list, ".mdf", scenario)
    
    for matrix_name in mat_list_mto.keys():
        mat_list_mto[matrix_name] = eb.available_matrix_identifier('FULL')
        create_matrix(matrix_id = mat_list_mto[matrix_name],
                  matrix_name= matrix_name.replace(" ","_"),
                  matrix_description = matrix_name)

    import_matrices_from_directory(mto_skims, mat_list_mto, ".mdf", scenario)
    
    for match in matching.keys():
        mto = mat_list_mto[match]
        hdr = matrix_list[matching[match]]
        spec = {
           "type": "MATRIX_CALCULATION",
           "result": mto,
           "expression":  hdr,
             "constraint": {
        "by_value": {
            "interval_min": 0,
            "interval_max": 0,
            "condition": "INCLUDE",
            "od_values": mto
        },
        "by_zone": None}
            }
        matrix_calculator(spec)
    min_exp = "("
    for matrix_name in mat_list_mto.keys():
        mat_id = mat_list_mto[matrix_name]
        min_exp = min_exp + mat_id + ".min."
        spec = {
           "type": "MATRIX_CALCULATION",
           "result": mat_id,
           "expression":  "9999",
             "constraint": {
        "by_value": {
            "interval_min": 0,
            "interval_max": 0,
            "condition": "INCLUDE",
            "od_values": mat_id
        },
        "by_zone": None}
            }
        matrix_calculator(spec)
    min_exp = min_exp[0:-5] + ")"

    spec = {
           "type": "MATRIX_CALCULATION",
           "result": min_transit_time,
           "expression":min_exp}
    matrix_calculator(spec)        
    spec = {
           "type": "MATRIX_CALCULATION",
           "result": min_transit_time,
           "expression":  "0",
        "constraint": {
        "by_value": {
            "interval_min": 9999,
            "interval_max": 9999,
            "condition": "INCLUDE",
            "od_values": min_transit_time
        },
        "by_zone": None}
            }
    matrix_calculator(spec)
    print "Finished Calculating OD Transit Travel Times"

def get_mc_spec(result, expression, agg_o, agg_d, con_o, con_d, con_val):
    specification = {
        "type": "MATRIX_CALCULATION",
        "result": result,
        "expression":  expression,
        "aggregation": {
            "origins": agg_o,
            "destinations": agg_d
        },
        "constraint": {
            "by_value": con_val,
            "by_zone": {
                "origins": con_o,
                "destinations": con_d
            }
        }
    }
    return specification

def MOE_fraction(out_file,numerator,denominator,zones_array):
    
    for origins in zones_array:
        spec = get_mc_spec(None, denominator,"+","+",origins,"all",None)
        bottom = matrix_calculator(spec)['result']
        spec = get_mc_spec(None, numerator,"+","+",origins,"all",None)
        top = float(matrix_calculator(spec)['result'])
        out_file.write("{0:.4f},".format(top/bottom))
    out_file.write("\n")
    
def roadMap(vdf):
    if vdf < 20: return "Freeway"
    else: return "Highway"
    
def myround(x, base=5):
    return int(base * round(float(x)/base))

def speedMapFreeway(speed):
    if speed < 35: return 35
    elif speed < 90:return myround(speed)
    else: return 90
    
def speedMapHighway(speed):
    if speed < 35: return 35
    elif speed < 80:return myround(speed)
    else: return 80
    
netCalcSpec = {
    "result": None,
    "expression": None,
    "aggregation": None,
    "selections": {
        "link": None
    },
    "type": "NETWORK_CALCULATION"
}

modes = {"sov":['trips_peak_hbw_auto1_nt', 'trips_peak_hbw_auto1_toll'],
        "hov2": ['trips_peak_hbw_auto2_nt_hov', 'trips_peak_hbw_auto2_nt_nh',
       'trips_peak_hbw_auto2_toll_hov', 'trips_peak_hbw_auto2_toll_nh'],
        "hov3": ['trips_peak_hbw_auto3_nt_hov', 'trips_peak_hbw_auto3_nt_nh',
       'trips_peak_hbw_auto3_toll_hov', 'trips_peak_hbw_auto3_toll_nh'],
        "Express Bus": ['trips_peak_hbw_transit_express_bus_walk','od_trips_peak_hbw_transit_express_bus_drive'],
        "Rapid Bus": ['trips_peak_hbw_transit_rapid_bus_walk','trips_peak_hbw_transit_rapid_bus_drive'],
        "Local Bus": ['trips_peak_hbw_transit_local_bus_walk','trips_peak_hbw_transit_local_bus_drive'],
        "Train":['od_trips_peak_hbw_transit_train_walk_access',
       'od_trips_peak_hbw_transit_train_transit_access',
       'od_trips_peak_hbw_transit_train_park_access',
       'od_trips_peak_hbw_transit_train_drop_off'],
        "Subway": ['od_trips_peak_hbw_transit_subway_walk_access',
       'od_trips_peak_hbw_transit_subway_transit_access',
       'od_trips_peak_hbw_transit_subway_park_access',
       'od_trips_peak_hbw_transit_subway_drop_off'],
        "walk": ['trips_peak_hbw_active_walk'],
        "bike": ['trips_peak_hbw_active_bike']}

truck_vols = ["@volau_htruck","@volau_mtruck","@volau_ltruck"]
auto_vols = ["@volau1_nt","@volau1_toll",\
             "@volau2_nt_nh","@volau2_nt_hov","@volau2_toll_nh","@volau2_toll_hov",\
            "@volau3_nt_nh","@volau3_nt_hov","@volau3_toll_nh","@volau3_toll_hov"]
sov_vols = ["@volau1_nt","@volau1_toll"]
hov2_vols = ["@volau2_nt_nh","@volau2_nt_hov","@volau2_toll_nh","@volau2_toll_hov"]
hov3_vols = ["@volau3_nt_nh","@volau3_nt_hov","@volau3_toll_nh","@volau3_toll_hov"]

In [4]:
#scenario folder names

#BAU
sc_2051_BAU = "2051\\Medium List\\BAU\\BAU2051_Updated_GGH_MTP_V106A3_CS222_20200703"
sc_2051_BAU_ogs = "2051\\Medium List\\BAU\\BAU_Updated_APTG_2051_GGH_MTP_V106A3_CS222_20200727"

#Network 1
sc_2051_N1_aptg = "2051\\Medium List\\Network 1\\N1_APTG_2051_GGH_MTP_V106A3_CS222_2020821"
sc_2051_N1_luv1 = "2051\\Medium List\\Network 1\\N1_LUV1_2051_GGH_MTP_V106A3_CS222_20200918"

#Network 2
sc_2051_N2_aptg = "2051\\Medium List\\Network 2\\N2_APTG_2051_GGH_MTP_V106A3_CS222_2020821"
sc_2051_N2_luv1 = "2051\\Medium List\\Network 2\\N2_LUV1_2051_GGH_MTP_V106A3_CS222_20200924"

#Network 3
sc_2051_N3_aptg = "2051\\Medium List\\Network 3\\N3_APTG_2051_GGH_MTP_V106A3_CS222_20200901"
sc_2051_N3_luv2 = "2051\\Medium List\\Network 3\\N3_LUV2_2051_GGH_MTP_V106A3_CS222_20201009"

#Network 4
sc_2051_N4_aptg = "2051\\Medium List\\Network 4\\N4_APTG_2051_GGH_MTP_V106A3_CS222_20200911"
sc_2051_N4_luv3 = "2051\\Medium List\\Network 4\\N4_LUV3_2051_GGH_MTP_V106A3_CS222_20201002"

#Network 5
sc_2051_N5_aptg = "2051\\Medium List\\Network 5\\N5_APTG_2051_GGH_MTP_V106A3_HS022_20200918"
sc_2051_N5_luv3 = "2051\\Medium List\\Network 5\\N5_LUV3_2051_GGH_MTP_V106A3_CS222_20201002"

## zone grouping definitions ## 
zones_pd1 = "gf1"
zones_airports = "ga1"
zones_ugc = "gb4"
zones_inner = "gc1"
zones_outer = "gc2"
zones_university = "gd1"
zones_employment = "ge1"
zones_hospitals = "gh1"
shed_central = "gs10;gs11"
rural_central = "gs11"
shed_east = "gs20;gs21"
rural_east = "gs21"
shed_midwest = "gs30;gs31"
rural_midwest = "gs31"
shed_southwest = "gs40;gs41"
rural_southwest = "gs41"
shed_west = "gs50;gs51"
rural_west = "gs51"
shed_north = "gs60;gs61"
rural_north = "gs61"
shed_niagara = "gs70;gs71"
rural_niagara = "gs71"
shed_urban = "gs10;gs20;gs30;gs40;gs50;gs60;gs70"
shed_rural = "gs11;gs21;gs31;gs41;gs51;gs61;gs71"
urban_lu = "gb2;gb3;gb4"

#auto occupancy
sov_oc = 1.2
hov2_oc = 2.2
hov3_oc = 3.3

#number of hours for transit assignment
num_hrs_am = 2
#num_hrs_md = 6

#Road costs and transit costs are half of what was provided by MTO in their estimates
#This is because I tag both sides of the link - and do not want to double count the cost of each project

roadCosts = {1: [5000000,4500000], # 2 lane widening
            2: [10000000,9000000], # 4 lane widening
            3: [15000000,13500000], # 6 lane widening
            4: [8375000,5750000], # 2 lane new construction
            5: [11875000,9250000], # 4 lane new construction
            6: [15375000,12750000], # 6 lane new construction
            7: [500000000,500000000], # 401 tunnel/overpass cost - Assumed it would be triple the cost of a normal 8 lane new construction
            8: [2500000,2500000], # Core/collector conversion cost
            9: [18875000,16250000]} # 8 lane new construction

transitCosts = {1: 12500000, # GO Train New Corridor (freight track only)
            2: 12500000, # GO Train New Corridor (no existing track)
            3: 5000000, # Go Train all day service and 15 minute
            4: 250000000, # Subway
            5: 62500000, # LRT
            6: 12000000, # BRT Lite
            7: 5000000000, # Missing Link Surplus cost
            8: 3500000, # GO train all day service
            9: 1500000, # GO train 15 minute service
            10: 5750000, #Ferry
            11: 5000000000, # Line 1/2 Capacity expansion surplus cost
            12: 100000000,#Kitchener Pearson realignment surplus cost
            13: 25000000}  #BRT Dedicated Facility

bridgeCost = {1: [29000000,29000000],# Assumes 10000 m2 bridge, using prestressed concrete, as these will be wide highway bridges
             2: [205000000,190000000], # 4 leg interchange
             3: [80000000,75000000], # 3 leg interchange
             4: [100000000,30000000]} # GO Station
#Based on parametric estimating guide. This is a surplus cost - not per km of bridge

In [5]:
pop_o_matrix_ogs = "mo19"
emp_o_matrix_ogs = "mo20"
senior_o_matrix_ogs = "mo21"
low_inc_o_matrix_ogs = "mo22"# number of low inc households
youth_o_matrix_ogs = "mo23"
pop_d_matrix_ogs = "md92"
emp_d_matrix_ogs = "md93"
low_income_zones_ogs = "mo24"

pop_o_matrix_luv1 = "mo25"
emp_o_matrix_luv1 = "mo26"
senior_o_matrix_luv1 = "mo27"
low_inc_o_matrix_luv1 = "mo28"# number of low inc households
youth_o_matrix_luv1 = "mo29"
pop_d_matrix_luv1 = "md94"
emp_d_matrix_luv1 = "md95"
low_income_zones_luv1 = "mo30"

pop_o_matrix_luv2 = "mo31"
emp_o_matrix_luv2 = "mo32"
senior_o_matrix_luv2 = "mo33"
low_inc_o_matrix_luv2 = "mo34"# number of low inc households
youth_o_matrix_luv2 = "mo35"
pop_d_matrix_luv2 = "md96"
emp_d_matrix_luv2 = "md97"
low_income_zones_luv2 = "mo36"

pop_o_matrix_luv3 = "mo37"
emp_o_matrix_luv3 = "mo38"
senior_o_matrix_luv3 = "mo39"
low_inc_o_matrix_luv3 = "mo40"# number of low inc households
youth_o_matrix_luv3 = "mo41"
pop_d_matrix_luv3 = "md98"
emp_d_matrix_luv3 = "md99"
low_income_zones_luv3 = "mo42"

hospital_d = "md90"
postsec_d = "md91"
parking_d = "md24"

truck_folder = r"\\tore-infs01\PWExternal\451010\281829_GGH\GGHM Runs\2051\Long List\BAU\MTO_2051_GGH_MTP_5002_V106A3_HS032_20200117\Assignable matrices"
tt_dir = r"\\tore-infs01\Models\10039241 - GGH Multimodal Transportation\HDR_transit_time_results\20200212_2051_BAU_HS - Peak"
tt_dir_op = r"\\tore-infs01\Models\10039241 - GGH Multimodal Transportation\HDR_transit_time_results\20200212_2051_BAU_HS - Off-Peak"
hdr_bau_peak = "mf160"
hdr_bau_offpeak = "mf170"
local_transit_fare = "mf180"

In [4]:
#All runs

scens = {"2051_BAU":[sc_2051_BAU,[401,402,403],[pop_o_matrix_ogs,emp_o_matrix_ogs,senior_o_matrix_ogs,
                                                low_inc_o_matrix_ogs,youth_o_matrix_ogs,pop_d_matrix_ogs,
                                                emp_d_matrix_ogs,low_income_zones_ogs]],
        "2051_BAU_OGS":[sc_2051_BAU_ogs,[404,405,406],[pop_o_matrix_ogs,emp_o_matrix_ogs,senior_o_matrix_ogs,
                                                low_inc_o_matrix_ogs,youth_o_matrix_ogs,pop_d_matrix_ogs,
                                                emp_d_matrix_ogs,low_income_zones_ogs]],
         "2051_N1_APTG":[sc_2051_N1_aptg,[501,502,503],[pop_o_matrix_ogs,emp_o_matrix_ogs,senior_o_matrix_ogs,
                                                low_inc_o_matrix_ogs,youth_o_matrix_ogs,pop_d_matrix_ogs,
                                                emp_d_matrix_ogs,low_income_zones_ogs]],
         "2051_N1_LUV1":[sc_2051_N1_luv1,[504,505,506],[pop_o_matrix_luv1,emp_o_matrix_luv1,senior_o_matrix_luv1,
                                                low_inc_o_matrix_luv1,youth_o_matrix_luv1,pop_d_matrix_luv1,
                                                emp_d_matrix_luv1,low_income_zones_luv1]],
        "2051_N2_APTG":[sc_2051_N2_aptg,[601,602,603],[pop_o_matrix_ogs,emp_o_matrix_ogs,senior_o_matrix_ogs,
                                                low_inc_o_matrix_ogs,youth_o_matrix_ogs,pop_d_matrix_ogs,
                                                emp_d_matrix_ogs,low_income_zones_ogs]],
         "2051_N2_LUV1":[sc_2051_N2_luv1,[604,605,606],[pop_o_matrix_luv1,emp_o_matrix_luv1,senior_o_matrix_luv1,
                                                low_inc_o_matrix_luv1,youth_o_matrix_luv1,pop_d_matrix_luv1,
                                                emp_d_matrix_luv1,low_income_zones_luv1]],
        "2051_N3_APTG":[sc_2051_N3_aptg,[701,702,703],[pop_o_matrix_ogs,emp_o_matrix_ogs,senior_o_matrix_ogs,
                                                low_inc_o_matrix_ogs,youth_o_matrix_ogs,pop_d_matrix_ogs,
                                                emp_d_matrix_ogs,low_income_zones_ogs]],
        "2051_N3_LUV2":[sc_2051_N3_luv2,[704,705,706],[pop_o_matrix_luv2,emp_o_matrix_luv2,senior_o_matrix_luv2,
                                                low_inc_o_matrix_luv2,youth_o_matrix_luv2,pop_d_matrix_luv2,
                                                emp_d_matrix_luv2,low_income_zones_luv2]],
        "2051_N4_APTG":[sc_2051_N4_aptg,[801,802,803],[pop_o_matrix_ogs,emp_o_matrix_ogs,senior_o_matrix_ogs,
                                                low_inc_o_matrix_ogs,youth_o_matrix_ogs,pop_d_matrix_ogs,
                                                emp_d_matrix_ogs,low_income_zones_ogs]],
        "2051_N4_LUV3":[sc_2051_N4_luv3,[804,805,806],[pop_o_matrix_luv3,emp_o_matrix_luv3,senior_o_matrix_luv3,
                                                low_inc_o_matrix_luv3,youth_o_matrix_luv3,pop_d_matrix_luv3,
                                                emp_d_matrix_luv3,low_income_zones_luv3]],
        "2051_N5_APTG":[sc_2051_N5_aptg,[901,902,903],[pop_o_matrix_ogs,emp_o_matrix_ogs,senior_o_matrix_ogs,
                                                low_inc_o_matrix_ogs,youth_o_matrix_ogs,pop_d_matrix_ogs,
                                                emp_d_matrix_ogs,low_income_zones_ogs]],
        "2051_N5_LUV3":[sc_2051_N5_luv3,[904,905,906],[pop_o_matrix_luv3,emp_o_matrix_luv3,senior_o_matrix_luv3,
                                                low_inc_o_matrix_luv3,youth_o_matrix_luv3,pop_d_matrix_luv3,
                                                emp_d_matrix_luv3,low_income_zones_luv3]]} #AM,MD,PM

In [6]:
scens = {"2051_N1_LUV1":[sc_2051_N1_luv1,[504,505,506],[pop_o_matrix_luv1,emp_o_matrix_luv1,senior_o_matrix_luv1,
                                                low_inc_o_matrix_luv1,youth_o_matrix_luv1,pop_d_matrix_luv1,
                                                emp_d_matrix_luv1,low_income_zones_luv1]],
         "2051_N2_LUV1":[sc_2051_N2_luv1,[604,605,606],[pop_o_matrix_luv1,emp_o_matrix_luv1,senior_o_matrix_luv1,
                                                low_inc_o_matrix_luv1,youth_o_matrix_luv1,pop_d_matrix_luv1,
                                                emp_d_matrix_luv1,low_income_zones_luv1]],
        "2051_N3_LUV2":[sc_2051_N3_luv2,[704,705,706],[pop_o_matrix_luv2,emp_o_matrix_luv2,senior_o_matrix_luv2,
                                                low_inc_o_matrix_luv2,youth_o_matrix_luv2,pop_d_matrix_luv2,
                                                emp_d_matrix_luv2,low_income_zones_luv2]],
        "2051_N4_LUV3":[sc_2051_N4_luv3,[804,805,806],[pop_o_matrix_luv3,emp_o_matrix_luv3,senior_o_matrix_luv3,
                                                low_inc_o_matrix_luv3,youth_o_matrix_luv3,pop_d_matrix_luv3,
                                                emp_d_matrix_luv3,low_income_zones_luv3]],
        "2051_N5_LUV3":[sc_2051_N5_luv3,[904,905,906],[pop_o_matrix_luv3,emp_o_matrix_luv3,senior_o_matrix_luv3,
                                                low_inc_o_matrix_luv3,youth_o_matrix_luv3,pop_d_matrix_luv3,
                                                emp_d_matrix_luv3,low_income_zones_luv3]]} #AM,MD,PM

In [7]:
#Matrix naming for inputs

auto_demand_pk = "TRIPS TOTAL AUTO peak"
auto_demand_offpk = "TRIPS TOTAL AUTO offpeak"

trip_am_auto1nt = "trips_am_peak_auto1_nt"
trip_am_auto1t = "trips_am_peak_auto1_toll" 
trip_am_auto2nth = "trips_am_peak_auto2_nt_hov"
trip_am_auto2ntnh = "trips_am_peak_auto2_nt_nh" 
trip_am_auto2th = "trips_am_peak_auto2_toll_hov" 
trip_am_auto2tnh ="trips_am_peak_auto2_toll_nh" 
trip_am_auto3nth ="trips_am_peak_auto3_nt_hov" 
trip_am_auto3ntnh ="trips_am_peak_auto3_nt_nh" 
trip_am_auto3th ="trips_am_peak_auto3_toll_hov" 
trip_am_auto3tnh ="trips_am_peak_auto3_toll_nh" 
trip_md_auto1nt ="trips_midday_auto1_nt" 
trip_md_auto1t ="trips_midday_auto1_toll"
trip_md_auto2nth ="trips_midday_auto2_nt_hov" 
trip_md_auto2ntnh ="trips_midday_auto2_nt_nh" 
trip_md_auto2th ="trips_midday_auto2_toll_hov" 
trip_md_auto2tnh ="trips_midday_auto2_toll_nh" 
trip_md_auto3nth ="trips_midday_auto3_nt_hov" 
trip_md_auto3ntnh ="trips_midday_auto3_nt_nh"
trip_md_auto3th ="trips_midday_auto3_toll_hov" 
trip_md_auto3tnh ="trips_midday_auto3_toll_nh" 

In [8]:
zones_array = ["all",zones_pd1,shed_central,shed_east,shed_midwest,shed_southwest,shed_west,shed_north,shed_niagara]
rural_array = [shed_rural,zones_pd1,rural_central,rural_east,rural_midwest,rural_southwest,rural_west,rural_north,rural_niagara]
zones_array_2 = [zones_pd1,shed_central,shed_east,shed_midwest,shed_southwest,shed_west,shed_north,shed_niagara]

In [9]:
ef_df = pd.read_csv(r"C:\Users\bcrane\Documents\GGH - Medium List\MOE\EmissionFactors_Input_20200917.csv")
shed_pth = r"C:\Users\bcrane\Documents\GGH - Medium List\MOE\NetworkJoin_Sheds"
out_file = r"C:\Users\bcrane\Documents\GGH - Medium List\20201019_ML_MOE_Exposure_N12345_LUV.csv"
shed_list = ["Regional","PD1","Central","East","Midwest","Southwest","West","North","Niagara"]

moef = open(out_file, 'w')
moef.write("Scen,MOE,MOE_ID," + ",".join(shed_list) + "\n")

for scen in scens.keys():
    print("Now processing {}".format(scen))
    scenario_folder = scens[scen][0]
    matrix_folder = r"\\tore-infs01\PWExternal\451010\281829_GGH\GGHM Runs" + "\\" + scenario_folder
    pop_o_matrix = scens[scen][2][0]
    
    sc_num_am = scens[scen][1][0]
    sc_num_md = scens[scen][1][1]

    #scenarios
    test_sc = eb.scenario(sc_num_am)
    test_sc_md = eb.scenario(sc_num_md)

    #networks
    ntwk = test_sc.get_network()
    ntwk_md = test_sc_md.get_network()
    
    linkList = []

    for link in ntwk.links():
        if link.volume_delay_func > 0 and link.volume_delay_func < 30 and ntwk.mode("I") in link.modes and link.auto_time > 0:
            auto_vol = 0.0
            for att in auto_vols:
                auto_vol += link[att]
            lm_truck_vol = link["@volau_mtruck"]+link["@volau_ltruck"]
            speed = link.length/link.auto_time*60
            linkList.append({"i":link.i_node.id,"j":link.j_node.id, "auto vol": auto_vol, "lm truck vol": lm_truck_vol,
                            "h truck vol": link["@volau_htruck"], "bus vol": link["@ca_trveh_l"],
                             "vdf": link.volume_delay_func,"length": link.length, "speed":speed})
            
    linkList_md = []
    for link in ntwk_md.links():
        if link.volume_delay_func > 0 and link.volume_delay_func < 30 and ntwk_md.mode("I") in link.modes and link.auto_time > 0:
            auto_vol = 0.0
            for att in auto_vols:
                auto_vol += link[att]
            lm_truck_vol = link["@volau_mtruck"]+link["@volau_ltruck"]
            speed = link.length/link.auto_time*60
            linkList_md.append({"i":link.i_node.id,"j":link.j_node.id, "auto vol": auto_vol, "lm truck vol": lm_truck_vol,
                            "h truck vol": link["@volau_htruck"], "bus vol": link["@ca_trveh_l"],
                             "vdf": link.volume_delay_func,"length": link.length, "speed":speed})
    
    link_df = pd.DataFrame.from_dict(linkList)
    link_df_md = pd.DataFrame.from_dict(linkList_md)
    
    if scen[5] == "B": short_name = scen[:8]
    else: short_name = scen[:7]
    
    zone_link_df = pd.read_csv(os.path.join(shed_pth,short_name + "_ZoneJoin.csv"))
    
    link_df["Road Type"] = link_df["vdf"].map(roadMap)
    link_df_md["Road Type"] = link_df_md["vdf"].map(roadMap)
    
    link_df_highway = link_df[link_df["Road Type"] == "Highway"]
    link_df_freeway = link_df[link_df["Road Type"] == "Freeway"]
    link_df_highway["AvgSpeed"] = link_df_highway["speed"].map(speedMapHighway)
    link_df_freeway["AvgSpeed"] = link_df_freeway["speed"].map(speedMapFreeway)
    link_df_2 = link_df_highway.append(link_df_freeway)

    link_df_highway_md = link_df_md[link_df_md["Road Type"] == "Highway"]
    link_df_freeway_md = link_df_md[link_df_md["Road Type"] == "Freeway"]
    link_df_highway_md["AvgSpeed"] = link_df_highway_md["speed"].map(speedMapHighway)
    link_df_freeway_md["AvgSpeed"] = link_df_freeway_md["speed"].map(speedMapFreeway)
    link_df_2_md = link_df_highway_md.append(link_df_freeway_md)
    
    emissions = []
    emissions_md = []
    for index, row in link_df_2.iterrows():
        speed = row.AvgSpeed
        roadType = row[u'Road Type']
        autoVol = row["auto vol"]
        busVol = row[u'bus vol']
        lmtruckVol = row[u'lm truck vol']
        htruckVol = row[u'h truck vol']
        length = row.length

        auto_pm25 = ef_df[(ef_df.RoadType2 == roadType) & (ef_df.AvgSpeed == speed) & (ef_df["Time of Day"] == "AM") & (ef_df.Vehicle == "Auto")]["PM2.5"].values[0]/1.60934
        bus_pm25 = ef_df[(ef_df.RoadType2 == roadType) & (ef_df.AvgSpeed == speed) & (ef_df["Time of Day"] == "AM") & (ef_df.Vehicle == "Bus")]["PM2.5"].values[0]/1.60934
        lmtruck_pm25 = ef_df[(ef_df.RoadType2 == roadType) & (ef_df.AvgSpeed == speed) & (ef_df["Time of Day"] == "AM") & (ef_df.Vehicle == "LM Truck")]["PM2.5"].values[0]/1.60934
        htruck_pm25 = ef_df[(ef_df.RoadType2 == roadType) & (ef_df.AvgSpeed == speed) & (ef_df["Time of Day"] == "AM") & (ef_df.Vehicle == "H Truck")]["PM2.5"].values[0]/1.60934

        auto_noxv = ef_df[(ef_df.RoadType2 == roadType) & (ef_df.AvgSpeed == speed) & (ef_df["Time of Day"] == "AM") & (ef_df.Vehicle == "Auto")]["NOX+VOC"].values[0]/1.60934
        bus_noxv = ef_df[(ef_df.RoadType2 == roadType) & (ef_df.AvgSpeed == speed) & (ef_df["Time of Day"] == "AM") & (ef_df.Vehicle == "Bus")]["NOX+VOC"].values[0]/1.60934
        lmtruck_noxv = ef_df[(ef_df.RoadType2 == roadType) & (ef_df.AvgSpeed == speed) & (ef_df["Time of Day"] == "AM") & (ef_df.Vehicle == "LM Truck")]["NOX+VOC"].values[0]/1.60934
        htruck_noxv = ef_df[(ef_df.RoadType2 == roadType) & (ef_df.AvgSpeed == speed) & (ef_df["Time of Day"] == "AM") & (ef_df.Vehicle == "H Truck")]["NOX+VOC"].values[0]/1.60934

        link_pm25 = length*(autoVol*auto_pm25 + busVol*bus_pm25 + lmtruckVol*lmtruck_pm25 + htruckVol*htruck_pm25)
        link_noxv = length*(autoVol*auto_noxv + busVol*bus_noxv + lmtruckVol*lmtruck_noxv + htruckVol*htruck_noxv)
        emissions.append({"i": row.i, "j": row.j, "pm25_am": link_pm25, "noxv_am": link_noxv})
    for index, row in link_df_2_md.iterrows():
        speed = row.AvgSpeed
        roadType = row[u'Road Type']
        autoVol = row["auto vol"]
        busVol = row[u'bus vol']
        lmtruckVol = row[u'lm truck vol']
        htruckVol = row[u'h truck vol']
        length = row.length

        auto_pm25 = ef_df[(ef_df.RoadType2 == roadType) & (ef_df.AvgSpeed == speed) & (ef_df["Time of Day"] == "MD") & (ef_df.Vehicle == "Auto")]["PM2.5"].values[0]/1.60934
        bus_pm25 = ef_df[(ef_df.RoadType2 == roadType) & (ef_df.AvgSpeed == speed) & (ef_df["Time of Day"] == "MD") & (ef_df.Vehicle == "Bus")]["PM2.5"].values[0]/1.60934
        lmtruck_pm25 = ef_df[(ef_df.RoadType2 == roadType) & (ef_df.AvgSpeed == speed) & (ef_df["Time of Day"] == "MD") & (ef_df.Vehicle == "LM Truck")]["PM2.5"].values[0]/1.60934
        htruck_pm25 = ef_df[(ef_df.RoadType2 == roadType) & (ef_df.AvgSpeed == speed) & (ef_df["Time of Day"] == "MD") & (ef_df.Vehicle == "H Truck")]["PM2.5"].values[0]/1.60934

        auto_noxv = ef_df[(ef_df.RoadType2 == roadType) & (ef_df.AvgSpeed == speed) & (ef_df["Time of Day"] == "MD") & (ef_df.Vehicle == "Auto")]["NOX+VOC"].values[0]/1.60934
        bus_noxv = ef_df[(ef_df.RoadType2 == roadType) & (ef_df.AvgSpeed == speed) & (ef_df["Time of Day"] == "MD") & (ef_df.Vehicle == "Bus")]["NOX+VOC"].values[0]/1.60934
        lmtruck_noxv = ef_df[(ef_df.RoadType2 == roadType) & (ef_df.AvgSpeed == speed) & (ef_df["Time of Day"] == "MD") & (ef_df.Vehicle == "LM Truck")]["NOX+VOC"].values[0]/1.60934
        htruck_noxv = ef_df[(ef_df.RoadType2 == roadType) & (ef_df.AvgSpeed == speed) & (ef_df["Time of Day"] == "MD") & (ef_df.Vehicle == "H Truck")]["NOX+VOC"].values[0]/1.60934

        link_pm25 = length*(autoVol*auto_pm25 + busVol*bus_pm25 + lmtruckVol*lmtruck_pm25 + htruckVol*htruck_pm25)
        link_noxv = length*(autoVol*auto_noxv + busVol*bus_noxv + lmtruckVol*lmtruck_noxv + htruckVol*htruck_noxv)
        emissions_md.append({"i": row.i, "j": row.j, "pm25_md": link_pm25, "noxv_md": link_noxv})

    emission_df = pd.DataFrame.from_dict(emissions)
    emission_df["i"] = emission_df["i"].map(int)
    emission_df["j"] = emission_df["j"].map(int)

    emission_df_md = pd.DataFrame.from_dict(emissions_md)
    emission_df_md["i"] = emission_df_md["i"].map(int)
    emission_df_md["j"] = emission_df_md["j"].map(int)

    emission_df = emission_df.merge(emission_df_md,left_on = ["i","j"], right_on = ["i","j"])
    
    shed_link_df = pd.read_csv(r"C:\Users\bcrane\Documents\GGH - Medium List\GGH_ZonesbyShed_20200820.csv")
    emission_df = emission_df.merge(zone_link_df,left_on = ["i","j"], right_on = ["INODE","JNODE"]).drop(labels = ["ID","INODE","JNODE"], axis = 1)
    emission_df =emission_df.merge(shed_link_df,left_on ="TAZ_NO", right_on = "TAZ").drop(labels = "TAZ", axis = 1)
    
    pop_mat = eb.matrix(pop_o_matrix).get_numpy_data(test_sc)
    indices = eb.matrix(pop_o_matrix).get_data(test_sc).indices
    pop_df = pd.DataFrame(data = pop_mat, index = indices[0]).reset_index().rename(columns = {"index": "TAZ", 0: "Pop"})
    emission_df = emission_df.merge(pop_df,left_on = "TAZ_NO", right_on = "TAZ").drop(labels = "TAZ", axis = 1)
    
    emission_df["noxv_am_pop"] = emission_df["noxv_am"]*emission_df["Pop"]
    emission_df["pm25_am_pop"] = emission_df["pm25_am"]*emission_df["Pop"]
    emission_df["noxv_md_pop"] = emission_df["noxv_md"]*emission_df["Pop"]
    emission_df["pm25_md_pop"] = emission_df["pm25_md"]*emission_df["Pop"]
    
    emission_shed_df = pd.pivot_table(data = emission_df, values = ["noxv_am_pop","pm25_am_pop","noxv_md_pop","pm25_md_pop"],
                                  index = "Shed", aggfunc = sum).reset_index()
    shed_link_df = shed_link_df.merge(pop_df,left_on="TAZ",right_on="TAZ")
    pop_shed_df = pd.pivot_table(data = shed_link_df, values = "Pop", index = "Shed", aggfunc =sum).reset_index()
    emission_shed_df = emission_shed_df.merge(pop_shed_df,left_on = "Shed", right_on = "Shed")
    emission_shed_df = emission_shed_df.append([{"Shed": "Regional", "noxv_am_pop": emission_shed_df["noxv_am_pop"].sum(),
                            "noxv_md_pop": emission_shed_df["noxv_md_pop"].sum(),
                            "pm25_am_pop": emission_shed_df["pm25_am_pop"].sum(),
                            "pm25_md_pop": emission_shed_df["pm25_md_pop"].sum(),
                            "Pop": emission_shed_df["Pop"].sum()}])
    
    clear_matrices(eb,[hdr_bau_peak, hdr_bau_offpeak,local_transit_fare],"FULL")
    matrix_list = {auto_demand_pk:"",
                    auto_demand_offpk:"",
                    trip_am_auto1nt:"",
                    trip_am_auto1t:"",
                    trip_am_auto2nth:"",
                    trip_am_auto2ntnh:"",
                    trip_am_auto2th:"",
                    trip_am_auto2tnh:"",
                    trip_am_auto3nth:"",
                    trip_am_auto3ntnh:"",
                    trip_am_auto3th:"",
                    trip_am_auto3tnh:"",
                    trip_md_auto1nt:"",
                    trip_md_auto1t:"",
                    trip_md_auto2nth:"",
                    trip_md_auto2ntnh:"",
                    trip_md_auto2th:"",
                    trip_md_auto2tnh:"",
                    trip_md_auto3nth:"",
                    trip_md_auto3ntnh:"",
                    trip_md_auto3th:"",
                    trip_md_auto3tnh:""}
    for matrix_name in matrix_list.keys():
        matrix_list[matrix_name] = eb.available_matrix_identifier('FULL')
        print matrix_name
        create_matrix(matrix_id = matrix_list[matrix_name],
                      matrix_name= matrix_name.replace("skim.","").replace("transit.","").replace("peak","pk").replace(" ","_").strip()[:40],
                      matrix_description = matrix_name)

    import_matrices_from_directory(matrix_folder, matrix_list, ".mdf",test_sc)

    auto_pk_hr_exp = matrix_list[trip_am_auto1nt] + "+" + matrix_list[trip_am_auto1t] + "+" + \
    matrix_list[trip_am_auto2nth] + "+" + matrix_list[trip_am_auto2ntnh] + "+" + matrix_list[trip_am_auto2th] + "+" + \
    matrix_list[trip_am_auto2tnh] + "+" + matrix_list[trip_am_auto3nth] + "+" + matrix_list[trip_am_auto3ntnh] + "+" + \
    matrix_list[trip_am_auto3th] + "+" + matrix_list[trip_am_auto3tnh]

    auto_offpk_hr_exp = matrix_list[trip_md_auto1nt] + "+" + matrix_list[trip_md_auto1t] + "+" + \
    matrix_list[trip_md_auto2nth] + "+" + matrix_list[trip_md_auto2ntnh] + "+" + matrix_list[trip_md_auto2th] + "+" + \
    matrix_list[trip_md_auto2tnh] + "+" + matrix_list[trip_md_auto3nth] + "+" + matrix_list[trip_md_auto3ntnh] + "+" + \
    matrix_list[trip_md_auto3th] + "+" + matrix_list[trip_md_auto3tnh]
    
    zones_array_match = {"all":"Regional",zones_pd1: "PD1",shed_central: "Central",shed_east:"East",
                         shed_midwest:"Midwest",shed_southwest:"Southwest",shed_west:"West",shed_north:"North",
                         shed_niagara:"Niagara"}
    shed_demand = []
    for origins in zones_array:
        spec = get_mc_spec(None,  matrix_list[auto_demand_pk],"+","+",origins,"all",None)
        pk_pd_dem = matrix_calculator(spec)["result"]
        spec = get_mc_spec(None, auto_pk_hr_exp,"+","+",origins,"all",None)
        pk_hr_dem = matrix_calculator(spec)["result"]
        spec = get_mc_spec(None,  matrix_list[auto_demand_offpk],"+","+",origins,"all",None)
        offpk_pd_dem = matrix_calculator(spec)["result"]
        spec = get_mc_spec(None, auto_offpk_hr_exp,"+","+",origins,"all",None)
        offpk_hr_dem = matrix_calculator(spec)["result"]
        shed_demand.append({"Shed": zones_array_match[origins],"pk_pd_dem": pk_pd_dem,"pk_hr_dem":pk_hr_dem,
                                                   "offpk_pd_dem": offpk_pd_dem, "offpk_hr_dem":offpk_hr_dem})
    shed_demand_df = pd.DataFrame.from_dict(shed_demand)
    emission_shed_df = emission_shed_df.merge(shed_demand_df,left_on="Shed",right_on="Shed")
    emission_shed_df["noxv_annual"] = 300*(emission_shed_df["noxv_am_pop"]*emission_shed_df["pk_pd_dem"]/emission_shed_df["pk_hr_dem"] +\
                                       emission_shed_df["noxv_md_pop"]*emission_shed_df["offpk_pd_dem"]/emission_shed_df["offpk_hr_dem"])
    emission_shed_df["pm25_annual"] = 300*(emission_shed_df["pm25_am_pop"]*emission_shed_df["pk_pd_dem"]/emission_shed_df["pk_hr_dem"] +\
                                       emission_shed_df["pm25_md_pop"]*emission_shed_df["offpk_pd_dem"]/emission_shed_df["offpk_hr_dem"])
    emission_shed_df["noxv_capita"] = emission_shed_df["noxv_annual"]/emission_shed_df["Pop"]
    emission_shed_df["pm25_capita"] = emission_shed_df["pm25_annual"]/emission_shed_df["Pop"]
    
    moef.write(scen + ",Exposure to NOx and VOC per capita within 500 metres of an expressway and highway (g (NOx+VOC) per capita per year),060203,")
    for origins in shed_list:
        noxv = emission_shed_df[emission_shed_df.Shed == origins]["noxv_capita"].values[0]
        moef.write("{0:.4f},".format(noxv))
    moef.write("\n")

    moef.write(scen + ",Exposure to PM2.5 per capita within 500 metres of an expressway and highway (g PM2_5 per capita per year),060204,")
    for origins in shed_list:
        pm25 = emission_shed_df[emission_shed_df.Shed == origins]["pm25_capita"].values[0]
        moef.write("{0:.4f},".format(pm25))
    moef.write("\n")
moef.close()

C:\Program Files\INRO\Emme\Emme 4\Emme-4.3.2\python-lib\win64\2.7\vendor\ipython\IPython\kernel\__main__.py:64: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\Program Files\INRO\Emme\Emme 4\Emme-4.3.2\python-lib\win64\2.7\vendor\ipython\IPython\kernel\__main__.py:65: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\Program Files\INRO\Emme\Emme 4\Emme-4.3.2\python-lib\win64\2.7\vendor\ipython\IPython\kernel\__main__.py:70: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_ind

Now processing 2051_N2_LUV1
trips_midday_auto2_toll_nh
trips_am_peak_auto2_nt_hov
trips_midday_auto3_toll_hov
trips_midday_auto2_nt_nh
trips_midday_auto3_nt_nh
trips_midday_auto2_toll_hov
trips_am_peak_auto1_nt
TRIPS TOTAL AUTO offpeak
trips_am_peak_auto2_nt_nh
trips_midday_auto1_toll
trips_midday_auto2_nt_hov
trips_am_peak_auto3_toll_hov
trips_midday_auto3_nt_hov
trips_am_peak_auto3_nt_hov
trips_am_peak_auto2_toll_nh
trips_midday_auto3_toll_nh
TRIPS TOTAL AUTO peak
trips_am_peak_auto3_nt_nh
trips_midday_auto1_nt
trips_am_peak_auto2_toll_hov
trips_am_peak_auto3_toll_nh
trips_am_peak_auto1_toll
Imported matrix mf8
Imported matrix mf17
Imported matrix mf7
Imported matrix mf22
Imported matrix mf2
Imported matrix mf9
Imported matrix mf20
Imported matrix mf15
Imported matrix mf14
Imported matrix mf18
Imported matrix mf12
Imported matrix mf21
Imported matrix mf19
Imported matrix mf10
Imported matrix mf11
Imported matrix mf4
Imported matrix mf6
Imported matrix mf1
Imported matrix mf13
Importe

In [94]:
emission_shed_df.to_csv(r"C:\Users\bcrane\Documents\GGH - Medium List\MOE\2051_BAU_pm25noxvoxExposure.csv")

In [104]:
scen = "2051_BAU_ogs"
shed_list = ["Regional","PD1","Central","East","Midwest","Southwest","West","North","Niagara"]

In [105]:
out_file = r"C:\Users\bcrane\Documents\GGH - Medium List\20201002_ML_MOE_Exposure_{}_test.csv".format(scen)

moef = open(out_file, 'w')
moef.write("Scen,MOE,MOE_ID," + ",".join(shed_list) + "\n")
#Connected Goal MOEs
#Objective 1

moef.write(scen + ",Exposure to NOx and VOC per capita within 500 metres of an expressway and highway (g (NOx+VOC) per capita per year),060203,")
for origins in shed_list:
    noxv = emission_shed_df[emission_shed_df.Shed == origins]["noxv_capita"].values[0]
    moef.write("{0:.4f},".format(noxv))
moef.write("\n")

moef.write(scen + ",Exposure to PM2.5 per capita within 500 metres of an expressway and highway (g PM2_5 per capita per year),060204,")
for origins in shed_list:
    pm25 = emission_shed_df[emission_shed_df.Shed == origins]["pm25_capita"].values[0]
    moef.write("{0:.4f},".format(pm25))
moef.write("\n")

    

In [99]:
emission_shed_df[emission_shed_df.Shed == "Regional"]["noxv_capita"].values[0]

5343841.1714475928

In [106]:
moef.close()